In [1]:
import pandas as pd
import os
import torch.utils.data as data

In [2]:
DATA_PATH = "../data/raw/"
MERGED_DATA_FILES = 'merged_data.csv'

### **Create tensor dataset**

In [3]:
class DatasetIDS2018(data.Dataset):
    def __init__(self, csv_file_name: str =os.path.join(DATA_PATH, MERGED_DATA_FILES), transform=None):
        self.data = pd.read_csv(csv_file_name)
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data_point = self.data[idx]
        data_label = self.label[idx]
        if self.transform:
            data_point = self.transform(data_point)
        return data_point, data_label
    
    def get_data_by_labels(self, labels: list) -> pd.DataFrame:
        mask = self.data['Label'].isin(labels)
        return pd.DataFrame(self.data[mask])

In [4]:
is_imported = True

In [5]:
if not is_imported:
    d = DatasetIDS2018()
    test_data = d.get_data_by_labels([
        'Brute Force -XSS',
        'DDOS attack-HOIC',
        'DDOS attack-LOIC-UDP',
        'DDoS attacks-LOIC-HTTP',
        'DoS attacks-GoldenEye',
        'DoS attacks-Hulk',
        ])

In [6]:
if not is_imported:
    test_data.groupby('Label')['Timestamp', ].count()

In [ ]:
def parse_dataset(dataset: pd.DataFrame, columns_to_drop: list, fixed_type, labels_emb: dict, labels_column_name: str) -> pd.DataFrame:
    for column in columns_to_drop:
        if column in list(dataset.columns):
            del dataset[column]
    dataset[labels_column_name] = dataset[labels_column_name].map(labels_emb)
    for column in dataset.columns:
        dataset[column] = pd.to_numeric(dataset[column], errors='coerce')
    return dataset